In [1]:
import re
import warnings
from typing import List
 
import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)
 
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
# load the model
MODEL_NAME = "fine_tuned_model_kbase_all_data"
 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto"
)
model = model.eval()
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# model genration config
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=generation_pipeline)
chain = ConversationChain(llm=llm)

In [6]:
# generate prompt
template = """
Current conversation:
{history}
Question: {input}
Answer:""".strip()
 
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

In [7]:
# add memory to the model
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)
 
chain = ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)
def kbasebot(text_input):
    res = chain.predict(input=text_input)
    return res

In [8]:
print(kbasebot("how to use KBase narrative?"))



> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:

Question: how to use KBase narrative?
Answer:


The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.



> Finished chain.
 To use KBase narrative, you can follow these steps:

1. Go to the KBase website and sign in with your Google or Globus account.
2. Click on the "Narrative" tab at the top of the page.
3. From there, you can create a new narrative by clicking on the green button that says "New Narrative".
4. Once you have created a new narrative, you will be taken to the narrative editor where you can start adding content such as text, images, videos, and more.
5. You can also add collaborators to your narrative if you want them to contribute or view it.
6. After making changes to your narrative, save it using the Save button located at the top right corner of the screen.
7. Your saved narrative will then appear under the Narratives section on the left-hand side of the page.
8. You can edit or delete this narrative later if needed.

By following these steps, you should now know how to use KBase narrative for creating and managing scientific narratives.


In [9]:
print(kbasebot("can you explain more for step 3?"))



> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:
Human: how to use KBase narrative?
AI:  To use KBase narrative, you can follow these steps:

1. Go to the KBase website and sign in with your Google or Globus account.
2. Click on the "Narrative" tab at the top of the page.
3. From there, you can create a new narrative by clicking on the green button that says "New Narrative".
4. Once you have created a new narrative, you will be taken to the narrative editor where you can start adding content such as text, images, videos, and more.
5. You can also add collaborators to your narrative if you want them to contribute or view it.
6. After making changes to your narrative, save it using the Save button located at the top right corner of the screen.
7. Your saved narrative will then appear under the Narratives section on the left-hand side of the page.
8. You can edit or delete this narrative later if needed.

By following these steps, you should now k